In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [2]:
checkpoint_path = "./eval_score"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)

# Load the model
model = AutoModelForSequenceClassification.from_pretrained(checkpoint_path)

In [3]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
device

'cpu'

In [8]:
id2label = {0: 3.0,
 1: 4.0,
 2: 4.5,
 3: 5.0,
 4: 5.5,
 5: 6.0,
 6: 6.5,
 7: 7.0,
 8: 7.5,
 9: 8.0,
 10: 8.5,
 11: 9.0}


In [ ]:
model.to(device)
def predictScore(question_text: str, answer_text: str):
    merged = question_text + tokenizer.sep_token + answer_text

    enc = tokenizer(
        merged,
        max_length=512,
        truncation=True,
        return_tensors="pt"  
    )
    enc = {k: v.to(device) for k, v in enc.items()}

    model.eval()
    with torch.no_grad():
        logits = model(**enc).logits.squeeze(0)   

    probs   = torch.softmax(logits, dim=-1).cpu().numpy()
    pred_id = int(probs.argmax())

    pred_label = id2label[pred_id]

    return pred_label, probs


In [20]:
question_text = "Young people are often influenced in their behaviours by others in the same age group. Some argue that peer pressure is important while others feel it has distinct disadvantages. Do the disadvantages of peer pressure outweigh the advantages?"
answer_text = (
    "In this day and age, peer pressure is considered as one of the most important factors affecting young people's behaviour. "
    "However, in this essay, I will discuss both advantages and disadvantages of this phenomenon.\n\n"
    "On the one hand, there are several drawbacks of peer pressure. To begin with, peer pressure can have a negative impact on young people. "
    "For example, if young people are affected by peer pressure, they may not be able to concentrate on their studies. As a result, they may be distracted from their studies. "
    "Moreover, young people may be influenced by other people whom they do not know. For instance, young people who are not well-prepared can be affected by other people who do not know them. "
    "Consequently, young people might be influenced by others who do not have enough knowledge.\n\n"
    "However, I believe that the advantages of peer pressure are more significant than the disadvantages. The first advantage is that peer pressure can improve young people's self-esteem. "
    "This is because peer pressure can increase the confidence of young people. Furthermore, young people can learn how to deal with peer pressure. "
    "As a consequence, young people will be able to cope with their peer pressure.\n\n"
    "In conclusion, although there are some disadvantages for peer pressure, I think that the benefits of this phenomenon outweigh the disadvantages. "
    "It is recommended that young people should be encouraged to learn how to handle peer pressure. In addition, young people should focus on their studies in order to avoid being influenced by other young people. "
    "This will help them to reduce their peer pressure and improve their self-esteem in the long run."
)

score, probs = predictScore(question_text, answer_text)

In [21]:
score

6.5

In [22]:
probs

array([1.1242753e-03, 1.6248821e-03, 8.0788095e-04, 9.7372878e-04,
       8.5943807e-03, 2.3198539e-01, 5.7853591e-01, 1.7079853e-01,
       4.1896510e-03, 5.0606049e-04, 2.7653226e-04, 5.8280176e-04],
      dtype=float32)